# VR CO2 Study - Data processing

This notebook does the following:

1. x
2. y
3. z

Input: input
Output: output

In [ ]:
# Imports
import os
if('notebooks' in os.getcwd()):
    os.chdir('..')
import json
import pandas as pd
from utils.enums import AirFiles, CO2Files
from utils.load_data import load_data_with_event_matching
import numpy as np
import datetime


In [ ]:
# Open file index JSON for reading
file_index = pd.read_json(os.path.join(os.getcwd(), 'temp/file_index.json'))
file_index = file_index.sort_index()

In [ ]:
# Example how to retrieve file names for a participant
participant_to_retrieve = 62
air_mask_file = file_index[0][participant_to_retrieve-1][AirFiles.MASK.value]
air_event_file = file_index[0][participant_to_retrieve-1][AirFiles.EVENT.value]

In [ ]:
print('Loading data for participant: ' + str(file_index[0][participant_to_retrieve-1]['id']))
participant_df = load_data_with_event_matching(air_mask_file, True, air_event_file)
print('Finished loading data for participant: ' + str(file_index[0][participant_to_retrieve-1]['id']))
participant_df

In [ ]:
# Load eye tracking data
air_eyetracking_file = file_index[0][participant_to_retrieve-1][AirFiles.EYE.value]
print('Loading eye data for participant: ' + str(file_index[0][participant_to_retrieve-1]['id']))
participant_eye_df = pd.read_csv(air_eyetracking_file)
print('Finished eye loading data for participant: ' + str(file_index[0][participant_to_retrieve-1]['id']))
participant_eye_df

In [ ]:
# Add eye tracking data

# TODO: CHECK IF EYE DATA FILE IS NOT EMPTY

#Some eye tracking files (participants 1-9) have empty 'TimestampUnix column'. Use their 'TimestampJ2000' to calculate unix column
if np.mean(participant_eye_df['TimestampUnix'].values)==0:
    print('TimestampUnix column is empty. Convert J2000 to Unix')
    j2000_timestamps = participant_eye_df['TimestampJ2000']
    unix_timestamps = []
    for j2000_timestamp in j2000_timestamps:
        utc_date_time = datetime.datetime(2000, 1, 1, 0, 0) + datetime.timedelta(milliseconds=j2000_timestamp)
        unix_timestamp = int(utc_date_time.timestamp()*1000)
        unix_timestamps.append(unix_timestamp)
    participant_eye_df['TimestampUnix']=unix_timestamps
else:
    print('TimestampUnix column is not empty.')
    
# TODO: SYNCHRONISE EYE DATA WITH MASK DF
    


In [ ]:
# Convert Unix
unix_timestamp = 1664453561.439    # Replace with your Unix timestamp
utc_date_time = datetime.datetime.utcfromtimestamp(unix_timestamp)
formatted_date_time = utc_date_time.strftime("%Y-%m-%d %H:%M:%S UTC")
print(formatted_date_time)

In [ ]:
# Convert J2000
j2000_timestamp = 717768761439
utc_date_time = datetime.datetime(2000, 1, 1, 0, 0) + datetime.timedelta(milliseconds=j2000_timestamp)
formatted_date_time = utc_date_time.strftime("%Y-%m-%d %H:%M:%S UTC")
print(formatted_date_time)

In [ ]:
# Add biopac data